In [5]:
import tensorflow as tf
import data_generator
from tensorflow.keras.layers import Conv2D, Flatten, Dense, Input, BatchNormalization, Dropout
from tensorflow.keras.models import Model
from keras.applications.vgg16 import VGG16

num_classes = 96

input_shape = (448, 448, 3)

base_model = VGG16(weights='imagenet', include_top=False, input_shape=input_shape)

for layer in base_model.layers:
    layer.trainable = False

x = base_model.output
x = BatchNormalization()(x)
x = Flatten()(x)
x = Dense(512, activation='relu')(x)
x = BatchNormalization()(x)
x = Dropout(0.35)(x)
x = Dense(512, activation='relu')(x)
x = BatchNormalization()(x)
x = Dropout(0.35)(x)

bbox_output = Dense(4, activation='linear', name='bboxes_output')(x)

labels_output = Dense(num_classes, activation='softmax', name='labels_output')(x)

model = Model(inputs=base_model.input, outputs=[bbox_output, labels_output])

def iou_metric(y_true, y_pred):
    x1_true, y1_true, x2_true, y2_true = tf.split(y_true, 4, axis=-1)
    x1_pred, y1_pred, x2_pred, y2_pred = tf.split(y_pred, 4, axis=-1)

    x1 = tf.maximum(x1_true, x1_pred)
    y1 = tf.maximum(y1_true, y1_pred)
    x2 = tf.minimum(x2_true, x2_pred)
    y2 = tf.minimum(y2_true, y2_pred)

    intersection = tf.maximum(x2 - x1, 0) * tf.maximum(y2 - y1, 0)
    area_true = (x2_true - x1_true) * (y2_true - y1_true)
    area_pred = (x2_pred - x1_pred) * (y2_pred - y1_pred)

    union = area_true + area_pred - intersection
    iou = intersection / tf.maximum(union, 1e-6)
    return iou

def iou_loss(y_true, y_pred):
    return 1 - iou_metric(y_true, y_pred)

model.compile(optimizer='rmsprop', loss={'bboxes_output': iou_loss, 'labels_output': 'categorical_crossentropy'}, metrics={'bboxes_output': iou_metric, 'labels_output': 'accuracy'}, jit_compile = False)

print(model.summary())

Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_1       │ (None, 448, 448,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1_conv1        │ (None, 448, 448,  │      1,792 │ input_layer_1[0]… │
│ (Conv2D)            │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1_conv2        │ (None, 448, 448,  │     36,928 │ block1_conv1[0][… │
│ (Conv2D)            │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1_pool         │ (None, 224, 224,  │          0 │ block1_conv2[0][… │
│ (MaxPooling2D)      │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block2_conv1        │ (None, 224, 224,  │     73,856 │ block1_pool[0][0] │
│ (Conv2D)            │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block2_conv2        │ (None, 224, 224,  │    147,584 │ block2_conv1[0][… │
│ (Conv2D)            │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block2_pool         │ (None, 112, 112,  │          0 │ block2_conv2[0][… │
│ (MaxPooling2D)      │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block3_conv1        │ (None, 112, 112,  │    295,168 │ block2_pool[0][0] │
│ (Conv2D)            │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block3_conv2        │ (None, 112, 112,  │    590,080 │ block3_conv1[0][… │
│ (Conv2D)            │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block3_conv3        │ (None, 112, 112,  │    590,080 │ block3_conv2[0][… │
│ (Conv2D)            │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block3_pool         │ (None, 56, 56,    │          0 │ block3_conv3[0][… │
│ (MaxPooling2D)      │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block4_conv1        │ (None, 56, 56,    │  1,180,160 │ block3_pool[0][0] │
│ (Conv2D)            │ 512)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block4_conv2        │ (None, 56, 56,    │  2,359,808 │ block4_conv1[0][… │
│ (Conv2D)            │ 512)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block4_conv3        │ (None, 56, 56,    │  2,359,808 │ block4_conv2[0][… │
│ (Conv2D)            │ 512)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block4_pool         │ (None, 28, 28,    │          0 │ block4_conv3[0][… │
│ (MaxPooling2D)      │ 512)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block5_conv1        │ (None, 28, 28,    │  2,359,808 │ block4_pool[0][0] │
│ (Conv2D)            │ 512)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block5_conv2        │ (None, 28, 28,    │  2,359,808 │ block5_conv1[0][

 Total params: 66,415,524 (253.36 MB)

 Trainable params: 51,697,764 (197.21 MB)

 Non-trainable params: 14,717,760 (56.14 MB)

None


In [4]:
image_dir_train = './output/train'
annotation_dir_train = './output/train'

image_dir_test = './output/test'
annotation_dir_test = './output/test'

batch_size = 8
epochs = 100

from collections import defaultdict
import os
data_path = './symbols.txt'
transforms = defaultdict()
with open(data_path, 'r') as txt_file:
    i = 0
    for line in txt_file:
        transforms[line.rstrip()] = i
        i += 1

train_generator = data_generator.DataGenerator(
    image_dir_train, annotation_dir_train,
    transforms=transforms,
    batch_size=batch_size,
    image_size=448
)

validation_generator = data_generator.DataGenerator(
    image_dir_test, annotation_dir_test,
    transforms=transforms,
    batch_size=batch_size,
    image_size=448
)

history = model.fit(
    train_generator,
    validation_data=validation_generator,
    epochs=epochs,
    steps_per_epoch=len(train_generator),
    validation_steps=len(validation_generator),
    compile=False
)

model.save_weights('./results/HCR-Net-Detection.h5')

Epoch 1/100


c:\Users\Панкратов\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


50/50 ━━━━━━━━━━━━━━━━━━━━ 514s 10s/step - bboxes_output_iou_metric: 0.0000e+00 - labels_output_accuracy: 0.0045 - loss: 6.7861 - val_bboxes_output_iou_metric: 0.0000e+00 - val_labels_output_accuracy: 0.0069 - val_loss: 6.1284
Epoch 2/100


c:\Users\Панкратов\AppData\Local\Programs\Python\Python312\Lib\contextlib.py:158: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(value)


AttributeError: 'NoneType' object has no attribute 'items'

In [ ]:
import matplotlib.pyplot as plt

plt.plot(history.history['labels_output_accuracy'])
plt.plot(history.history['val_labels_output_accuracy'])
plt.title('Model Accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper left')
plt.show()

plt.plot(history.history['labels_output_loss'])
plt.plot(history.history['val_labels_output_loss'])
plt.title('Model Classification Loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper left')
plt.show()

plt.plot(history.history['bboxes_output_iou_metric'])
plt.plot(history.history['val_bboxes_output_iou_metric'])
plt.title('Model IoU Metric')
plt.ylabel('IoU')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper left')
plt.show()

plt.plot(history.history['bboxes_output_loss'])
plt.plot(history.history['val_bboxes_output_loss'])
plt.title('Model Bounding Box Loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper left')
plt.show()